In [1]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_sample_weight
import pickle
import numpy as np
np.set_printoptions (threshold=None, edgeitems=None, linewidth=None, suppress=None)

In [2]:
import torch

In [3]:
def standardize(col):
   return (col - col.mean()) / col.std()

In [4]:
dataset_path = "/home/local/ASURITE/speddira/dev/traffic_sense_net/city_scale/processed_datasets/2024-2-16_1915hours_4jun_600_win_600twin.csv"

df = pd.read_csv(dataset_path)

data_types = {
    'incident_edge': 'object',  # Replace 'Column_Name1' with the actual column name
    'incident_lane': 'object'  # Replace 'Column_Name2' with the actual column name
}

df = pd.read_csv(dataset_path,dtype=data_types)

/tmp/ipykernel_1654890/4036539136.py:3: DtypeWarning: Columns (63,68) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dataset_path)


In [5]:
df = df[df["accident_label"] == True]
df['label'] = None

incident_category = ["stalled_vehicle","multi_vehicle_collision"]
for category in incident_category:
# Find rows where incident_edge is in the current set of edges
    mask = df['incident_type'] == category
# Set the appropriate label column to 1 for these rows
    if category ==  "stalled_vehicle":
        df.loc[mask, 'label'] = 0
    else:
        df.loc[mask, 'label'] = 1

In [6]:
df = df.fillna(-1)

Y = df["label"]

X = df.drop(["incident_edge","incident_start_time","incident_type","accident_id","accident_duration","incident_lane","accident_label","label"],axis=1)
X = X.apply(standardize)

X = X.drop(["Unnamed: 0","step"],axis=1)
X= X.values
Y= Y.values

/tmp/ipykernel_1654890/3492477022.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(-1)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)


In [8]:
clf = TabNetClassifier(n_d=64, n_a=64, momentum=0.3, n_steps=5, optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    scheduler_params = {"gamma": 0.95,
                     "step_size": 20},
    scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15)

/home/local/ASURITE/speddira/miniconda3/envs/cobra/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [9]:
clf.fit(
  X_train, y_train,num_workers = 10,max_epochs=80
)

/home/local/ASURITE/speddira/miniconda3/envs/cobra/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.45636 |  0:00:36s
epoch 1  | loss: 0.26494 |  0:01:15s
epoch 2  | loss: 0.11904 |  0:01:56s
epoch 3  | loss: 0.02226 |  0:02:37s
epoch 4  | loss: 0.01345 |  0:03:21s
epoch 5  | loss: 0.01587 |  0:04:04s
epoch 6  | loss: 0.00888 |  0:04:40s
epoch 7  | loss: 0.00738 |  0:05:24s
epoch 8  | loss: 0.00603 |  0:06:08s
epoch 9  | loss: 0.00626 |  0:06:50s
epoch 10 | loss: 0.00527 |  0:07:33s
epoch 11 | loss: 0.00674 |  0:08:15s
epoch 12 | loss: 0.0062  |  0:08:54s
epoch 13 | loss: 0.00686 |  0:09:41s
epoch 14 | loss: 0.00329 |  0:10:25s
epoch 15 | loss: 0.00308 |  0:11:10s
epoch 16 | loss: 0.00417 |  0:11:53s
epoch 17 | loss: 0.00324 |  0:12:36s
epoch 18 | loss: 0.00242 |  0:13:15s
epoch 19 | loss: 0.00477 |  0:13:58s
epoch 20 | loss: 0.00242 |  0:14:42s
epoch 21 | loss: 0.00306 |  0:15:24s
epoch 22 | loss: 0.00288 |  0:16:02s
epoch 23 | loss: 0.00211 |  0:16:47s
epoch 24 | loss: 0.00273 |  0:17:31s
epoch 25 | loss: 0.0026  |  0:18:11s
epoch 26 | loss: 0.00274 |  0:18:54s
e

In [10]:
saving_path_name = "tab_net_4_jun_severity"
saved_filepath = clf.save_model(saving_path_name)

Successfully saved model at tab_net_4_jun_severity.zip


In [11]:
X= X_test
Y= y_test
y_pred = clf.predict(X)
y_pred = list(y_pred)
y_test = list(Y)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("precision: %.2f%%" % (precision * 100.0))
print("f1: %.2f%%" % (f1 * 100.0))
print("recall: %.2f%%" % (recall * 100.0))

print("*****************************************************************")


Accuracy: 99.96%
precision: 100.00%
f1: 99.97%
recall: 99.95%
*****************************************************************
